<a href="https://colab.research.google.com/github/Espebaum/Gemma2b-it-Write-My-Paper/blob/main/gemma_write_my_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U bitsandbytes # 양자화를 위한 bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 6.8 

In [3]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

### 파일을 로드합니다.
https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=90 의 데이터를 사용했습니다.

JSON 파일로 된 데이터를 pandas dataframe으로 변환하고, 그것을 SFTTrainer가 받아들일 수 있는 Dataset 형태로 변환합니다.

In [4]:
import pandas as pd
import json # JSON 파일 처리

# 1. JSON 파일 로드
path_0 = '/content/drive/MyDrive/Colab_Notebooks/Gemma2/paper_train/paper_train_00.json'
path_1 = '/content/drive/MyDrive/Colab_Notebooks/Gemma2/paper_train/paper_train_01.json'
path_2 = '/content/drive/MyDrive/Colab_Notebooks/Gemma2/paper_train/paper_train_02.json'
path_3 = '/content/drive/MyDrive/Colab_Notebooks/Gemma2/paper_train/paper_train_03.json'

with open(path_0, 'r', encoding='utf-8') as f:
    data_0 = json.load(f)

with open(path_1, 'r', encoding='utf-8') as f:
    data_1 = json.load(f)

with open(path_2, 'r', encoding='utf-8') as f:
    data_2 = json.load(f)

with open(path_3, 'r', encoding='utf-8') as f:
    data_3 = json.load(f)

# 데이터를 담을 리스트 초기화
documents = []

# 2. 데이터 파싱
for item in data_0[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_entire', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_entire', [])])
    }
    documents.append(doc)

for item in data_1[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_entire', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_entire', [])])
    }
    documents.append(doc)

for item in data_2[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_entire', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_entire', [])])
    }
    documents.append(doc)

for item in data_3[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_entire', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_entire', [])])
    }
    documents.append(doc)

for item in data_0[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_section', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_section', [])])
    }
    documents.append(doc)

for item in data_1[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_section', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_section', [])])
    }
    documents.append(doc)

for item in data_2[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_section', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_section', [])])
    }
    documents.append(doc)


for item in data_3[0]['data']:
    doc = {
        'category': item.get('ipc', ''),
        'title': item.get('title', ''),
        'summary': ' '.join([summary['summary_text'] for summary in item.get('summary_section', [])]),
        'original': ' '.join([summary['orginal_text'] for summary in item.get('summary_section', [])])
    }
    documents.append(doc)


# pandas 데이터프레임으로 변환
df = pd.DataFrame(documents)
print(df.shape) # 약 (320000, 4)
df.head()

(288174, 4)


,category,title,summary,original
0,인문학,중세시대의 對句 학습과 문학 교육,"對句의 영역은 다양하고, 그 원리는 짝짓기이다. 하나의 텍스트가 같은 분포도를 보이...","짐작할 수 있는 것처럼, 對句의 영역은 음운, 단어, 어구, 시편의 차원에서 다양하..."
1,인문학,일제하 토지회수운동의 전개과정 - 전남 무안군 하의도의 사례 -,하의도에 상륙한 신한공사 사원과 경관 등은 소작료를 강제 징수하고자 2개조로 나뉘어...,다음과 같다. 하의도에 상륙한 신한공사 사원과 경관 등은 소작료를 강제 징수하기 위...
2,사회과학,코스닥시장 가격제한폭 변화가 주가지수 변동성에 미치는 영향,코스닥시장에서 가격제한폭 변화가 이루어진 1998. 5. 25의 경우를 대상으로 검...,코스닥시장에서 가격제한폭 변화가 이루어진 1998. 5. 25의 경우를 대상으로 분...
3,의약학,치료 중인 암환아와 치료 종료 암환아의 사회심리적 적응에 관한 비교 연구,"소아암 환아의 사회심리적 적응의 문제의 위험요소에는 아동의 특성, 질병과 치료관련 ...",소아암 환아의 사회심리적 적응의 문제를 야기할 수 있는 위험요소로 인구학적 변수를 ...
4,의약학,리스페리돈과 올란자핀을 사용한 정신분열병 환자 치료 패턴의 변이 조사,"본 연구는 국내의 정신분열병 치료의 변이 정도를 파악하고자, 병원의 진료 실태, 다...",정신분열병의 약물치료를 비롯한 정신과 치료에 있어서 다양한 임상적 결정을 내릴 때에...


## 세트 분할

###데이터 프레임을 훈련, 검증 세트로 분할하여 SFTTrainer에서 파인 튜닝 시 일정 훈련 step마다 검증 스텝을 추가하려고 했지만 메모리 문제로 인해 실패했습니다.

### 검증 스텝을 추가하는 것이 메모리 문제와 어떤 관계가 있는지는 알아내지 못했습니다. 문제를 해결하지 못하여 훈련 스텝만 진행했습니다.

In [5]:
from sklearn.model_selection import train_test_split

# 데이터프레임을 훈련, 검증세트로 분할, 실제로는 사용하지 못함
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [6]:
# pandas 데이터프레임을 Trainer에서 사용하기 위한 dataset 모양으로 변경
dataset = Dataset.from_pandas(df)

# tr_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# print("훈련 세트 크기:", train_df.shape)
# print("검증 세트 크기:", val_df.shape)
# val_dataset[0]

print(dataset.shape)
dataset[0]

(288174, 4)


{'category': '인문학',
 'title': '중세시대의 對句 학습과 문학 교육',
 'summary': '對句의 영역은 다양하고, 그 원리는 짝짓기이다. 하나의 텍스트가 같은 분포도를 보이는 것은 자연스럽다.',
 'original': '짐작할 수 있는 것처럼, 對句의 영역은 음운, 단어, 어구, 시편의 차원에서 다양하게 이뤄지고 있으며, 그 원리는 짝짓기이다. 하나의 텍스트는 음운으로부터 텍스트 자체까지에 이르는 위계적 양상을 보이므로 對句의 영역 또한 같은 분포도를 보이는 것은 퍽 자연스럽다. 문제는 텍스트의 전 영역에 걸쳐 짝을 만들어야 한다는 강박증이다. 왜 한시학은 짝짓기에 집착하는가?'}

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
# https://www.kaggle.com/models/google/gemma-2 에서 Notebook을 만드는 것으로 input에 Model을 추가할 수 있습니다.

BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")
# device_map="auto"로 설정하여 gpu를 자동으로 할당해주었습니다. (깊게 건들기 어려운 부분인 것 같습니다...)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [9]:
doc_sum = dataset[0]['summary']
doc_ori = dataset[0]['original']
print(doc_sum)
print(doc_ori)

對句의 영역은 다양하고, 그 원리는 짝짓기이다. 하나의 텍스트가 같은 분포도를 보이는 것은 자연스럽다.
짐작할 수 있는 것처럼, 對句의 영역은 음운, 단어, 어구, 시편의 차원에서 다양하게 이뤄지고 있으며, 그 원리는 짝짓기이다. 하나의 텍스트는 음운으로부터 텍스트 자체까지에 이르는 위계적 양상을 보이므로 對句의 영역 또한 같은 분포도를 보이는 것은 퍽 자연스럽다. 문제는 텍스트의 전 영역에 걸쳐 짝을 만들어야 한다는 강박증이다. 왜 한시학은 짝짓기에 집착하는가?


### https://huggingface.co/google/gemma-2-2b-it 에서도 Gemma2의 모델 카드를 확인할 수 있습니다.

* pipeline 활용법이 제시되어 있습니다.

---
```
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",  # replace with "mps" to run on a Mac device
)

messages = [
    {"role": "user", "content": "Who are you? Please, answer in pirate-speak."},
]

outputs = pipe(messages, max_new_tokens=256)
assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)

# Ahoy, matey! I be Gemma, a digital scallywag, a language-slingin' parrot of the digital seas. I be here to help ye with yer wordy woes, answer yer questions, and spin ye yarns of the digital world.  So, what be yer pleasure, eh? 🦜
```
---

In [11]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512
)
# gemma2 에서 제공하는 파이프라인

In [15]:
chat = [
    {
        "role": "user",
        "content": "Write a hello world program"
    },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 위에서 만든 pipe를 활용, 여기서 매개변수를 자유롭게 조정 가능
outputs = pipe(
    prompt,
    do_sample=True,         # 모델이 확률적으로 샘플링을 통해 텍스트를 생성하도록 설정
    # temperature=0.2,        # 확률분포의 평탄도, 낮은 값(0.2)은 모델이 더 결정적으로 응답을 생성하게 하고 높은 값은 응답이 더 무작위적으로 생성
    # top_k=50,               # 확률적으로 상위 50개의 단어만 고려하여 다음 단어를 샘플링하는 방식
    # top_p=0.95,             # 확률 분포에서 상위 95%에 해당하는 단어들만 선택되도록 설정
    add_special_tokens=True # 모델이 특별 토큰을 처리할 수 있도록 설정
)

# pipe 사용법인 것 같다.
print(outputs[0]["generated_text"][len(prompt):])

```python
print("Hello, world!")
```

This Python program will print the string "Hello, world!" to the console.

**Explanation:**

* `print()` is a built-in Python function that prints the given argument to the console.
* `"Hello, world!"` is the string that we want to print.
* `\n` is a newline character, which is added to ensure proper spacing.

**How to run the program:**

1. Save this file as `hello_world.py` (or any other name with a `.py` extension).
2. Open a terminal window or command prompt.
3. Navigate to the directory where you saved the file.
4. Run the following command:

```
python hello_world.py
```

**Output:**

This command will execute the Python script and print the following output to the console:

```
Hello, world!
```

**Note:**

* The `print()` function can take multiple arguments, which will be separated by commas.
* You can use different formatting options in the `print()` function, such as `name` and `value` to format the output differently.


In [12]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 더 길게 써주세요:\n\n{}".format(doc_sum)
    }
]

# apply_chat_template() 은 Huggingface에서 제공하는 함수로 모델에 매개변수로 사용하기 위한
# prompt를 편리하게 제작하도록 지원하는 함수입니다. (https://huggingface.co/docs/transformers/main/en/chat_templating)

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

'<bos><start_of_turn>user\n다음 글을 더 길게 써주세요:\n\n對句의 영역은 다양하고, 그 원리는 짝짓기이다. 하나의 텍스트가 같은 분포도를 보이는 것은 자연스럽다.<end_of_turn>\n<start_of_turn>model\n'

In [13]:
# pipe 사용 예시

outputs = pipe(
    prompt,
    do_sample=True,         # 모델이 확률적으로 샘플링을 통해 텍스트를 생성하도록 설정
    temperature=0.2,        # 확률분포의 평탄도, 낮은 값(0.2)은 모델이 더 결정적으로 응답을 생성하게 하고 높은 값은 응답이 더 무작위적으로 생성
    top_k=50,               # 확률적으로 상위 50개의 단어만 고려하여 다음 단어를 샘플링하는 방식
    top_p=0.95,             # 확률 분포에서 상위 95%에 해당하는 단어들만 선택되도록 설정
    add_special_tokens=True # 모델이 특별 토큰을 처리할 수 있도록 설정
)

# pipe 사용법인 것 같다.
print(outputs[0]["generated_text"][len(prompt):])

**더 긴 글:**

對句의 영역은 다양하고, 그 원리는 짝짓기이다. 짝짓기는 두 개의 문장을 서로 연결하는 어법으로, 하나의 텍스트가 같은 분포도를 보이는 것을 의미한다. 

짝짓기는 문법적으로 정확한 연결어법이 아니지만, 자연스러운 표현을 만들기 위해 사용하는 일종의 문법적 표현이다. 짝짓기는 일반적으로 문장의 시작과 끝에 사용되는 경우가 많으며, 주로 서로 다른 분포를 보이는 문장을 연결하는 데 사용된다.

짝짓기는 문장의 구조를 변화시키거나, 주제와 내용을 강조하는 데 사용될 수 있다. 또한, 짝짓기는 특정 분포를 보여주는 문장을 표현하는 데 사용될 수 있다.


## 양자화



In [14]:
lora_config = LoraConfig(
    r=6, # 저차원 행렬의 랭크(rank) 정의
    lora_alpha = 8, # LoRA에서 사용되는 스케일링(Scaling) 팩터
    lora_dropout = 0.05, # 드롭아웃으로 과적합 방지
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    # LoRA 적용을 위한 모델 내의 특정 레이어(모듈) 지정
    task_type="CAUSAL_LM", # task_type은 LoRA가 적용되는 작업의 유형을 지정함
    # 여기서는 CAUSAL_LM(Causal Language Modeling)을 나타낸다.
)

bnb_config = BitsAndBytesConfig(
    # 입력값을 4 bit로 변환
    load_in_4bit=True,
    # 모델을 4 bit으로 양자화
    bnb_4bit_quant_type="nf4",
    # 4 bit 계산에 사용될 데이터 유형, 4비트 부동소수점(bfloat16), 4비트 정수(uint8)
    bnb_4bit_compute_dtype=torch.float16
)

In [15]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
# model.add_adapter(lora_config, adapter_name="adapter_1")

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# 위에서 pandas Dataframe으로부터 만들었던 dataset을 매개변수로 받는다.

def generate_prompt(example):
  prompt_list = []
  for i in range(len(example['summary'])):
    prompt_list.append(r"""<bos><start_of_turn>user
    다음 글을 더 길게 써주세요:
    {}<end_of_turn>
    <start_of_turn>model
    {}<end_of_turn><eos>""".format(example['summary'][i], example['original'][i]))
  return prompt_list

In [17]:
train_data = dataset
print(generate_prompt(train_data[0:1])[0])

<bos><start_of_turn>user
    다음 글을 더 길게 써주세요:
    對句의 영역은 다양하고, 그 원리는 짝짓기이다. 하나의 텍스트가 같은 분포도를 보이는 것은 자연스럽다.<end_of_turn>
    <start_of_turn>model
    짐작할 수 있는 것처럼, 對句의 영역은 음운, 단어, 어구, 시편의 차원에서 다양하게 이뤄지고 있으며, 그 원리는 짝짓기이다. 하나의 텍스트는 음운으로부터 텍스트 자체까지에 이르는 위계적 양상을 보이므로 對句의 영역 또한 같은 분포도를 보이는 것은 퍽 자연스럽다. 문제는 텍스트의 전 영역에 걸쳐 짝을 만들어야 한다는 강박증이다. 왜 한시학은 짝짓기에 집착하는가?<end_of_turn><eos>


In [18]:
from transformers import EarlyStoppingCallback

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    # train_dataset=val_dataset,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="/content/drive/MyDrive/Colab_Notebooks/Gemma2/write_my_paper_output/gemma_WriteMyPaper",
        num_train_epochs = 1,
        max_steps=5000,
        per_device_train_batch_size=1, # 각 GPU마다 처리되는 샘플의 개수
        gradient_accumulation_steps=4, # 늘리면 메모리가 줄어듬 (한 스텝에 batchsize * acc_steps = 8개 샘플 처리)
        optim="paged_adamw_8bit",
        warmup_ratio=0.03,
        learning_rate=2e-4,
        fp16=True, # fp16=True,
        logging_steps=100,
        # eval_strategy="steps",  # 검증을 위한 평가 전략 설정
        # eval_steps=1500,  # 매 2000 스텝마다 검증 수행
        # save_steps=1500,  # eval_steps의 배수로 설정
        # save_total_limit=1,  # 체크포인트 저장 제한
        # load_best_model_at_end=True,  # 가장 성능 좋은 모델을 로드
        # metric_for_best_model="eval_loss",  # 최상의 모델을 선택할 때 사용할 지표
        # greater_is_better=False,  # 지표가 낮을수록 좋은 경우(예: 손실)
        push_to_hub=False,
        report_to=None,
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
    # callbacks=[EarlyStoppingCallback(
        # early_stopping_patience=3,  # patience: 개선이 없는 스텝 수
        # early_stopping_threshold=0.01  # 개선으로 간주되는 최소 변화율
    # )]
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/288174 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
import os

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
trainer.train()

Step,Training Loss
100,2.432000
200,1.884900
300,1.794400
400,1.783200
500,1.811000
600,1.771800
700,1.775700
800,1.763900
900,1.750800
1000,1.745100


TrainOutput(global_step=5000, training_loss=1.7152511688232421, metrics={'train_runtime': 8677.1357, 'train_samples_per_second': 2.305, 'train_steps_per_second': 0.576, 'total_flos': 1.1538290661520589e+17, 'train_loss': 1.7152511688232421, 'epoch': 0.06940251375904835})

## Hugging Face에 모델 배포

In [ ]:
!git lfs install
!git clone https://huggingface.co/gyopark/gemma-2-2b-it-WriteMyPaper

import shutil

Git LFS initialized.
Cloning into 'gemma-2-2b-it-WriteMyPaper'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (4/4), 1.12 KiB | 1.13 MiB/s, done.


In [ ]:
# 로컬 경로
source_dir = "/content/drive/MyDrive/Colab_Notebooks/Gemma2/write_my_paper_output/gemma_WriteMyPaper"  # gemma_ThinkLink 모델이 저장된 폴더
destination_dir = "/content/gemma-2-2b-it-WriteMyPaper"  # Hugging Face에서 클론한 폴더


In [ ]:
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

'/content/gemma-2-2b-it-WriteMyPaper'

In [ ]:
import os

os.chdir("/content/gemma-2-2b-it-WriteMyPaper")  # Git 저장소로 이동
!git lfs track "*.safetensors"
!git lfs track "tokenizer.json"

"*.safetensors" already supported
Tracking "tokenizer.json"


In [ ]:
!git status

!git config --global user.email "vocops5000@gmail.com"
!git config --global user.name "gyopark"

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   README.md

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	adapter_1/
	adapter_config.json
	adapter_model.safetensors
	optimizer.pt
	rng_state.pth
	scheduler.pt
	special_tokens_map.json
	tokenizer.json
	tokenizer.model
	tokenizer_config.json
	trainer_state.json
	training_args.bin

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add .gitattributes  # LFS 관련 파일 추가
!git add --all
!git commit -m "Add Fine-Tuned WriteMyPaper Model"
!git push

[main f3f5fee] Add Fine-Tuned WriteMyPaper Model
 14 files changed, 2468 insertions(+), 3 deletions(-)
 rewrite README.md (69%)
 create mode 100644 adapter_1/adapter_config.json
 create mode 100644 adapter_1/adapter_model.safetensors
 create mode 100644 adapter_config.json
 create mode 100644 adapter_model.safetensors
 create mode 100644 optimizer.pt
 create mode 100644 rng_state.pth
 create mode 100644 scheduler.pt
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer.json
 create mode 100644 tokenizer.model
 create mode 100644 tokenizer_config.json
 create mode 100644 trainer_state.json
 create mode 100644 training_args.bin
Uploading LFS objects: 100% (8/8), 96 MB | 24 MB/s, done.
Enumerating objects: 18, done.
Counting objects: 100% (18/18), done.
Delta compression using up to 8 threads
Compressing objects: 100% (16/16), done.
Writing objects: 100% (16/16), 8.08 KiB | 2.69 MiB/s, done.
Total 16 (delta 0), reused 0 (delta 0), pack-reused 0
To https://huggingface.c

## 이제 https://huggingface.co/gyopark/gemma-2-2b-it-WriteMyPaper 를 통해 파인튜닝된 모델의 모델카드를 확인할 수 있습니다.